In [10]:
import json
import pandas as pd
import numpy as np
import re
import copy
import nltk
import stemming
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.stem import PorterStemmer
from stemming.porter2 import stem
from ngram import *
import csv

In [11]:
data = [] 
cf = open('LVbars_reviewdf.csv','r')

file = csv.DictReader(cf)
#print(file.fieldnames)

for x in file:
    line = {'business_id':x['business_id'],'date':x['date'],'stars':x['stars'],'text':x['text']}
    data.append(line)

cf.close()

In [12]:
#pd.DataFrame(data[0:100]).to_csv('LVbars_reviewdf_less.csv',index=False)

In [13]:
dat_night=data

In [14]:
dat_night_5star=[]
dat_night_4star=[]
dat_night_3star=[]
dat_night_2star=[]
dat_night_1star=[]
for i in range(len(dat_night)):
    if dat_night[i]['stars']=='5.0':
        dat_night_5star.append(dat_night[i])
    elif dat_night[i]['stars']=='4.0':
        dat_night_4star.append(dat_night[i])
    elif dat_night[i]['stars']=='3.0':
        dat_night_3star.append(dat_night[i])
    elif dat_night[i]['stars']=='2.0':
        dat_night_2star.append(dat_night[i])
    else:
        dat_night_1star.append(dat_night[i])



In [21]:
reviewslist = copy.deepcopy(dat_night_1star)
REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\$)|(\*)|(\%)|(\=)|(\#)|(\&)|(\~)|(\@)")#[^\P{P}-]+
REPLACE_WITH_SPACE = re.compile("(\n)|(\-)|(\/)|(\d)")

def preprocess_reviews(reviews):
    reviews = REPLACE_NO_SPACE.sub("", reviews)
    reviews = REPLACE_WITH_SPACE.sub(" ", reviews)
    return reviews

reviewsTEXT_clean = copy.deepcopy(reviewslist)

for ind in range(len(reviewslist)):
    texts = ''
    texts = reviewslist[ind]['text']
    texts = texts.lower()
    texts = re.sub('n\'t',' not', texts)
    texts = re.sub('isnt','isn\'t', texts)
    texts = re.sub('wasnt','wasn\'t', texts)
    texts = re.sub('werent','weren\'t', texts)
    texts = re.sub('dont','don\'t', texts)
    texts = re.sub('doesnt','doesn\'t', texts)
    texts = re.sub('didnt','didn\'t', texts)
    texts = re.sub('hasnt','hasn\'t', texts)
    texts = re.sub('havent','haven\'t', texts)
    texts = re.sub('hadnt','hadn\'t', texts)
    texts = re.sub('mightnt','mightn\'t', texts)
    texts = re.sub('shouldnt','shouldn\'t', texts)
    texts = re.sub('isn','isn\'t', texts)
    texts = re.sub('wasn','wasn\'t', texts)
    texts = re.sub('weren','weren\'t', texts)
    texts = re.sub('don','don\'t', texts)
    texts = re.sub('doesn','doesn\'t', texts)
    texts = re.sub('didn','didn\'t', texts)
    texts = re.sub('hasn','hasn\'t', texts)
    texts = re.sub('haven','haven\'t', texts)
    texts = re.sub('hadn','hadn\'t', texts)
    texts = re.sub('mightn','mightn\'t', texts)
    texts = re.sub('shouldn','shouldn\'t', texts)
    
    texts = re.sub('n\'t',' not', texts)
    
    #add NOT_
    pattern = r'\.|\;|\!|\?|\,|\)|\(|\:|\'|\"|\%'
    list_text=re.split(pattern,texts)
    
    sent=''
    for i in range(len(list_text)):
        list_text[i] = re.sub('\+','', list_text[i])
        list_text[i] = re.sub('\*','', list_text[i])
        list_text[i] = re.sub('\$','', list_text[i])
        list_text[i] = re.sub('\[','', list_text[i])
        list_text[i] = re.sub('\]','', list_text[i])
        list_text[i] = re.sub('\%','', list_text[i])
        list_text[i] = re.sub('\\\\',' ', list_text[i])
        matchObj1 = re.search(r'(.*)not (.*)',list_text[i])
        matchObj2 = re.search(r'(.*) never (.*)',list_text[i])
        
        if matchObj1 != None :
            sub=re.sub(r' ', " NOT_", ' '+matchObj1.group(2))
            list_text[i] = re.sub(matchObj1.group(2) ,sub, list_text[i])
            list_text[i] = re.sub(r'not ','', list_text[i],1)
            #list_text[i] = re.sub(',',' ', matchObj1.group(3))
            sent = sent + list_text[i] + ' '
        
        elif matchObj2 != None :
            sub=re.sub(r' ', " NOT_", ' '+matchObj2.group(2))
            list_text[i] = re.sub(matchObj2.group(2) ,sub, list_text[i])
            list_text[i] = re.sub(r'never ','', list_text[i],1)
            #list_text[i] = re.sub(',',' ', matchObj1.group(3))
            sent = sent + list_text[i] + ' '
        
        else:
            sent = sent + list_text[i] + ' '
   
    #print(sent)
    #print(texts)
    #print(texts)
    text_clean=preprocess_reviews(sent)
    #print(text_clean)
    st = text_clean
    st2 = " ".join([stem(word) for word in st.split(" ")])
    text_clean = st2
    
    reviewsTEXT_clean[ind]['text']=text_clean
    reviewsTEXT_clean[ind]['text_split']=text_clean.split()
    reviewsTEXT_clean[ind]['freq']=nltk.FreqDist(reviewsTEXT_clean[ind]['text_split'])   
    reviewsTEXT_clean[ind]['bigrams']=generate_ngrams(text_clean, 2)
    reviewsTEXT_clean[ind]['bigrams_freq']=nltk.FreqDist(reviewsTEXT_clean[ind]['bigrams'])
    
#reviewsTEXT_clean[0]

In [22]:
#select words due to frequency order
AllFreq = reviewsTEXT_clean[0]['freq']
for ind in range(len(reviewsTEXT_clean)-1):
    freqs = reviewsTEXT_clean[ind+1]['freq']
    AllFreq.update(freqs)
    
wordList = sorted(AllFreq.items(),key=lambda item:item[1],reverse=True)

#choose the important ones--No.1-100
WORDS=[x[0] for x in wordList]
WORDS=WORDS[0:1000]

In [23]:
#stop_words = set(stopwords.words('english')) 
stop_words = {'who', 'how', 'him', 'can', 'than', 'these', 'your', 'the', 'while', 'don', 'of', 'on', 'had', 'there', "you've", 'that', 'having', 'himself', "mustn't", 'same', 'are', "won't", 'then', 'itself', 'doing', 'from', 'both', 'where', 'wouldn', 'me', 'off', 'because', 'isn', "you'd", 'whom', 'mustn', 'is', 'themselves', 'no', 'very', 'up', 'd', 'ma', 'yours', 'been', 'ain', 'will', 'a', 'most', 'did', 'with', 'o', 'this', 'during', 'i', "mightn't", "isn't", 'being', 'couldn', 'them', 'not', 'such', 'her', 'some', 'only', "didn't", 'should', 'after', 'our', 'down', 'here', 'about', 'herself', "hadn't", 'but', 'he', 'an', 'am', 't', 'they', 'again', 'll', 've', 'didn', 'into', 'needn', 're', 'nor', "couldn't", 'above', 'all', "should've", 'm', 'other', 'below', "she's", 'just', 'between', 'hasn', 'own', 'yourselves', 'until', 'too', 'which', "shouldn't", 's', "it's", 'his', 'y', 'to', 'over', 'hadn', "shan't", 'does', 'weren', 'shouldn', 'under', "aren't", 'be', "don't", 'any', 'or', "haven't", 'she', 'aren', 'against', 'we', 'in', 'ourselves', 'have', 'won', "wasn't", 'wasn', 'you', 'what', 'mightn', "weren't", 'doesn', 'hers', 'myself', 'shan', 'before', 'more', "wouldn't", 'were', 'each', "doesn't", 'through', 'for', "hasn't", 'by', 'now', 'do', 'has', 'those', 'few', "you'll", 'once', 'it', 'their', 'further', "you're", 'my', 'at', 'when', 'yourself', 'why', 'as', 'was', 'and', 'out', "needn't", 'if', 'haven', 'its', 'theirs', "that'll", 'so', 'ours'}
ps = PorterStemmer()
stop_words_stem=[]
for w in stop_words:
    stop_words_stem.append(ps.stem(w))
negWords=set(["wouldn't",'isn','wasn',"weren't", "haven't", "hasn't", "couldn't", "isn't", 'doesn','hasn',"mustn't", 'mightn', 'shan', 'no', "wasn't",'aren', "didn't", "hadn't","don't",'nor',"won't",'weren',"doesn't","needn't", 'shouldn',"mightn't","shan't", 'wouldn',"shouldn't",'hadn'])
negWords_stem=[]
for w in negWords:
    negWords_stem.append(ps.stem(w))
d=set(['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','à','{sigh}'])
stopWords_stem=set(stop_words_stem)-set(negWords_stem)
stopWords_stem.update(d)
stopWords_stem.add('was')

WORDS=[x for x in WORDS if x not in stopWords_stem]


print('high-frequency words:')
print(WORDS[0:100])

high-frequency words:
['this', 'food', 'NOT_the', 'NOT_to', 'us', 'place', 'order', 'no', 'servic', 'get', 'would', 'time', 'one', 'drink', 'like', 'ask', 'go', 'NOT_a', 'wait', 'tabl', 'NOT_', 'said', 'could', 'minut', 'came', 'got', 'bar', 'manag', 'back', 'good', 'NOT_and', 'told', 'restaur', 'peopl', 'NOT_it', 'even', 'friend', 'want', 'come', 'went', 'tri', 'look', 'server', 'room', 'night', 'vega', 'say', 'NOT_be', 'NOT_in', 'took', 'NOT_for', 'realli', 'walk', 'experi', 'check', 'NOT_i', 'NOT_of', 'first', 'NOT_this', 'NOT_even', 'hour', 'bartend', 'take', 'NOT_that', 'bad', 'NOT_back', 'way', 'waitress', 'anoth', 'custom', 'call', 'NOT_go', 'make', 'club', 'better', 'ever', 'worst', 'well', 'give', 'two', 'NOT_have', 'NOT_on', 'also', 'still', 'NOT_was', 'left', 'hotel', 'has', 'horribl', 'NOT_at', 'seat', 'great', 'his', 'eat', 'guy', 'disappoint', 'rude', 'know', 'around', 'staff']


In [24]:
dict_wc_uni={}
for i in range(len(wordList)):
    if wordList[i][0] in WORDS:
        dict_wc_uni[wordList[i][0]]=wordList[i][1]

In [25]:
'''
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wc = WordCloud(background_color = 'White',width=400,height=300,scale=4,max_font_size=100)
wc.generate_from_frequencies(dict_wc)

plt.figure(figsize=(20,12), dpi=1200)
plt.imshow(wc)
plt.axis("off")
plt.show()
'''

'\nfrom wordcloud import WordCloud\nimport matplotlib.pyplot as plt\n\nwc = WordCloud(background_color = \'White\',width=400,height=300,scale=4,max_font_size=100)\nwc.generate_from_frequencies(dict_wc)\n\nplt.figure(figsize=(20,12), dpi=1200)\nplt.imshow(wc)\nplt.axis("off")\nplt.show()\n'

In [26]:
words=list(dict_wc_uni.keys())
frequency=list(dict_wc_uni.values())
df_star1 = pd.DataFrame(np.nan,index=range(len(words)), columns=['loc'])
df_star1['words']=words
df_star1['frequency']=frequency
df_star1=df_star1.loc[:,['words','frequency']]
from pandas import DataFrame
DataFrame.to_csv(df_star1,"wc_uni_star1.csv",index=False)

In [27]:
#select biwords due to frequency order
BiFreq = reviewsTEXT_clean[0]['bigrams_freq']
for ind in range(len(reviewsTEXT_clean)-1):
    freqs = reviewsTEXT_clean[ind+1]['bigrams_freq']
    BiFreq.update(freqs)
    
biwordList = sorted(BiFreq.items(),key=lambda item:item[1],reverse=True)

#choose the important ones--No.1-100
BIWORDS=[x[0] for x in biwordList]
BIWORDS=BIWORDS[0:1000]

In [34]:
BIWORDS[0:20]

['it was',
 'of the',
 'we were',
 'in the',
 'to the',
 'i was',
 'this place',
 'and the',
 'and i',
 'on the',
 'at the',
 'for the',
 'for a',
 'the food',
 'it s',
 'to get',
 'to be',
 'i m',
 'we had',
 'had to']

In [31]:
dict_wc_bi={}
for i in range(len(biwordList)):
    if biwordList[i][0] in BIWORDS:
        dict_wc_bi[biwordList[i][0]]=biwordList[i][1]

In [32]:
biwords=list(dict_wc_bi.keys())
frequency=list(dict_wc_bi.values())
df_star1 = pd.DataFrame(np.nan,index=range(len(biwords)), columns=['loc'])
df_star1['biwords']=biwords
df_star1['frequency']=frequency
df_star1=df_star1.loc[:,['biwords','frequency']]
from pandas import DataFrame
DataFrame.to_csv(df_star1,"wc_bi_star1.csv",index=False)